In [1]:
from selenium import webdriver
import urllib.request
from PIL import Image
import pandas as pd

option = webdriver.ChromeOptions()

browser = webdriver.Chrome(executable_path='/Users/blakeduncan/Documents/chromedriver', options=option)

In [9]:
def thesportsdb_login(username, password):
    browser.get("https://www.thesportsdb.com/")
    browser.find_element_by_xpath("//*[@id='header']/nav/div/div[2]/ul/li[5]/a[1]").click()
    browser.find_element_by_xpath("//*[@id='feature']/div/div[3]/div/form/div[1]/input").send_keys(username)
    browser.find_element_by_xpath("//*[@id='feature']/div/div[3]/div/form/div[2]/input").send_keys(password)
    browser.find_element_by_xpath("//*[@id='rememberme']").click()
    browser.find_element_by_xpath("//*[@id='feature']/div/div[3]/div/form/div[4]/input").click()

def add_event(league, date, starttime, season, hometeam, awayteam, week):
    for i in range(len(league)):
        browser.get(f"https://www.thesportsdb.com/edit_event_add.php?l={league[i]}")
        browser.find_element_by_xpath("//*[@id='datepicker']").send_keys(date[i])
        browser.find_element_by_xpath("//*[@id='starttime']").send_keys(starttime[i])
        browser.find_element_by_xpath("//*[@id='season']").send_keys(season[i])
        browser.find_element_by_xpath("//*[@id='hometeam']").send_keys(hometeam[i])
        browser.find_element_by_xpath("//*[@id='awayteam']").send_keys(awayteam[i])
        browser.find_element_by_xpath("//*[@id='round']").send_keys(week[i])
        browser.find_element_by_xpath("//*[@id='submit']").click()

def add_score(league, date, starttime, season, hometeam, awayteam, homescore, awayscore, week):
    for i in range(len(league)):
        browser.get(f"https://www.thesportsdb.com/edit_event_add.php?l={league[i]}")
        browser.find_element_by_xpath("//*[@id='datepicker']").send_keys(date[i])
        browser.find_element_by_xpath("//*[@id='starttime']").send_keys(starttime[i])
        browser.find_element_by_xpath("//*[@id='season']").send_keys(season[i])
        browser.find_element_by_xpath("//*[@id='hometeam']").send_keys(hometeam[i])
        browser.find_element_by_xpath("//*[@id='awayteam']").send_keys(awayteam[i])
        browser.find_element_by_xpath("//*[@id='homescore']").send_keys(homescore[i])
        browser.find_element_by_xpath("//*[@id='awayscore']").send_keys(awayscore[i])
        browser.find_element_by_xpath("//*[@id='round']").send_keys(week[i])
        browser.find_element_by_xpath("//*[@id='submit']").click()

def add_player(team, fullname, dob, position, nationality, height, weight):
    for i in range(len(team)):
        browser.get(f"https://www.thesportsdb.com/edit_player_add.php?t={team[i]}")
        browser.find_element_by_xpath("//*[@id='fullname']").send_keys(fullname[i])
        browser.find_element_by_xpath("//*[@id='datepicker']").send_keys(dob[i])
        browser.find_element_by_xpath("//*[@id='position']").send_keys(position[i])
        browser.find_element_by_xpath("//*[@id='countries']").send_keys(nationality[i])
        browser.find_element_by_xpath("//*[@id='submit']").click()
        browser.find_element_by_xpath("//*[@id='feature']/div/div/div[3]/a[1]").click()
        browser.find_element_by_xpath("//*[@id='feature']/div/div/div[1]/a[4]/button").click()
        browser.find_element_by_xpath("//*[@id='height']").send_keys(height[i])
        browser.find_element_by_xpath("//*[@id='weight']").send_keys(weight[i])
        browser.find_element_by_xpath("//*[@id='submit']").click()

def retrieve_image(player, image_url):
    urllib.request.urlretrieve(
        image_url,
        f"/Users/blakeduncan/Downloads/{player}.jpg")
    return Image.open(f"/Users/blakeduncan/Downloads/{player}.jpg").convert('RGB')

def crop_image(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    if img_height > img_width:
        coordinates = pil_img.crop((0, 0, img_width, crop_height))
    else:
        coordinates = pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))
    return coordinates

def crop_max_square(pil_img):
    return crop_image(pil_img, min(pil_img.size), min(pil_img.size))

def save_player_thumbnail(player, image_url):
    img = retrieve_image(player, image_url)
    im_thumb = crop_max_square(img).resize((700, 700), Image.LANCZOS)
    return im_thumb.save(f"/Users/blakeduncan/Downloads/{player}.jpg")
        
def upload_player_thumbnail(player, image_url):
    for i in range(len(player)):
        try:
            save_player_thumbnail(player[i], image_url[i])
            search = player[i].replace(" ", "+")
            browser.get(f"https://www.thesportsdb.com/search.php?s={search}")
            try:
                browser.find_element_by_xpath("//*[@id='feature']/div/div/div[3]/a/img").click()
                browser.find_element_by_xpath("//*[@id='feature']/div/div/div[1]/a[1]/img").click()
                try:
                    browser.find_element_by_xpath("/html/body/form/input").send_keys(f"/Users/blakeduncan/Downloads/{player[i]}.jpg")
                    browser.find_element_by_xpath("/html/body/form/p[3]/input").click()
                except:
                    continue
            except:
                continue
        except:
            continue

In [3]:
thesportsdb_login("cardinaldiehard25", "HHtW3b64iW4SAR@")

In [5]:
event_csv = pd.read_csv("/Users/blakeduncan/Downloads/event.csv")

league = event_csv.league.tolist()
date = event_csv.date.tolist()
starttime = event_csv.starttime.tolist()
season = event_csv.season.tolist()
hometeam = event_csv.hometeam.tolist()
awayteam = event_csv.awayteam.tolist()
week = event_csv.week.tolist()

add_event(league, date, starttime, season, hometeam, awayteam, week)

In [7]:
score_csv = pd.read_csv("/Users/blakeduncan/Downloads/score.csv")

league = score_csv.league.tolist()
date = score_csv.date.tolist()
starttime = score_csv.starttime.tolist()
season = score_csv.season.tolist()
hometeam = score_csv.hometeam.tolist()
awayteam = score_csv.awayteam.tolist()
homescore = score_csv.homescore.tolist()
awayscore = score_csv.awayscore.tolist()
week = score_csv.week.tolist()

add_score(league, date, starttime, season, hometeam, awayteam, homescore, awayscore, week)

In [7]:
player_csv = pd.read_csv("/Users/blakeduncan/Downloads/pro14.csv")

team = player_csv.team.tolist()
fullname = player_csv.fullname.tolist()
dob = player_csv.dob.tolist()
position = player_csv.position.tolist()
nationality = player_csv.nationality.tolist()
height = player_csv.height.tolist()
weight = player_csv.weight.tolist()

add_player(team, fullname, dob, position, nationality, height, weight)

In [7]:
image_url_csv = pd.read_csv("/Users/blakeduncan/Downloads/cardiff4.csv")

player = image_url_csv.player.tolist()
image_url = image_url_csv.image_url.tolist()

upload_player_thumbnail(player, image_url)

HTTPError: HTTP Error 403: Forbidden